In [1]:
import pandas as pd
import matplotlib as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
//anaconda3/envs/sklearn-env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//anaconda3/envs/sklearn-env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//anaconda3/envs/sklearn-env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//anaconda3/envs/sklearn-env/lib/pytho

In [2]:
real = pd.read_csv('data/True.csv')
fake = pd.read_csv('data/Fake.csv')

In [3]:
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [117]:
real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 5 columns):
title      21417 non-null object
text       21417 non-null object
subject    21417 non-null object
date       21417 non-null object
fake       21417 non-null int64
dtypes: int64(1), object(4)
memory usage: 836.7+ KB


In [5]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
title      23481 non-null object
text       23481 non-null object
subject    23481 non-null object
date       23481 non-null object
dtypes: object(4)
memory usage: 733.9+ KB


In [15]:
real.loc[real.text == ' '].index

Int64Index([8970], dtype='int64')

In [12]:
fake = fake.drop(fake.loc[fake.text == ' '].index)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


In [118]:
fake.loc[fake.text.str.contains('Reuters')]

,title,text,subject,date
115,"Maine Voters Tell Trump To Go F*ck Himself, E...",Republicans should be downright afraid to try ...,News,"November 8, 2017"
265,"Trump Walks Into Church In Puerto Rico, Start...","Before departing for Puerto Rico, Donald Trump...",News,"October 3, 2017"
348,"Trump’s UN Speech May Have Been Terrible, But...",By now you would be more than aware of Preside...,News,"September 20, 2017"
350,Trump Just Became First President In Modern H...,Donald Trump has been fundraising like crazy f...,News,"September 19, 2017"
417,BREAKING: Under Trump’s Leadership FEMA Could...,"After Hurricane Harvey devastated Houston, Tex...",News,"September 7, 2017"
582,Trump Is Now Talking About Going To War With ...,"Before dawn on Wednesday, the feds raided the ...",News,"August 11, 2017"
730,Trump Gets HUMILIATED By His Own Joint Chiefs...,It appears Donald Trump jumped the gun when he...,News,"July 27, 2017"
783,POLL: Trump Is The Least Popular President Of...,"Donald Trump may think he s the best, but it l...",News,"July 22, 2017"
802,Russian Lawyer At Trump Jr.’s Meeting Has Wor...,The whole world knows about the bombshell repo...,News,"July 21, 2017"
885,Trump Jr.’s Latest Move Proves That Russia In...,Proving that the biggest obstacle to his presi...,News,"July 10, 2017"


In [18]:
real.loc[real.text.str.contains('Reuters')].count()/real.count()

title      0.998179
text       0.998179
subject    0.998179
date       0.998179
dtype: float64

In [ ]:
fake_drop.loc[fake_drop.text.str.contains('Reuters')].count()/fake_drop.count()

In [20]:
# dropping rows that have urls as text and date, real's dates look fine
fake_drop = fake.drop(index=[9358,15507,15508,18933])

In [40]:
# Give labels to data before combining
fake_drop['fake'] = 1
real['fake'] = 0

In [84]:
combined = pd.concat([fake_drop, real])

In [ ]:
## this stuff might not be needed...
my_stop_words = set(stopwords.words('english')).union('reuters')
combined.title = combined.title.apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
combined.text = combined.text.apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
combined.text.apply(word_tokenize)
def preprocess(df: pd.DataFrame, column:str, stop_words: set) -> pd.DataFrame:
    df[column] = df[column].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    df[column] = df[column].apply(lambda x: x.lower())
    return df

In [57]:
tokenizer = Tokenizer(num_words=2500, lower=True,split=' ')

In [58]:
tokenizer.fit_on_texts(combined['text'].values)

In [92]:
X = tokenizer.texts_to_sequences(combined['text'].values)
X = pad_sequences(X)

In [108]:
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(1, name='out_layer'))
model.add(Activation('sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

//anaconda3/envs/sklearn-env/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
//anaconda3/envs/sklearn-env/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  import sys


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 6416, 128)         320000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 200)               263200    
_________________________________________________________________
out_layer (Dense)            (None, 1)                 201       
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 583,401
Trainable params: 583,401
Non-trainable params: 0
_________________________________________________________________
None


In [100]:
y = combined['fake']

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [112]:
model.fit(X_train, y_train, batch_size=batch_size, epochs = 1, validation_split=0.2,\
          callbacks=[EarlyStopping(monitor='val_loss')])

Train on 26936 samples, validate on 6734 samples
Epoch 1/1
  160/26936 [..............................] - ETA: 2:02:17 - loss: 0.6774 - accuracy: 0.7375

KeyboardInterrupt: 